In [4]:
import sys
sys.path.insert(0, '../Core-scripts/')

from parse_and_prepare import ProteinProteinInteractionClassifier as ppi
import file_readers as fr
import prediction as pred

In [5]:
import pickle
import os
import re
from sklearn.metrics import roc_curve, auc
import numpy as np
from gensim.models import word2vec
import logging

In [6]:
dros_strict_real = pickle.load(open('../../Results/Drosophila/drosophila_mentions_strict_real.pkl', 'rb'))
dros_gen_real = pickle.load(open('../../Results/Drosophila/drosophila_mentions_gen_real.pkl', 'rb'))
dros_be_real = pickle.load(open('../../Results/Drosophila/drosophila_mentions_be_real.pkl', 'rb'))
random_seeds = [144, 235, 905, 2895, 3462, 4225, 5056, 5192, 7751, 7813]

In [4]:
for seed in random_seeds:
    real_tr_te_name = 'Drosophila/train_test/dros_tr_te_split_' + str(seed)
    train_data, b, c, d = pred.manual_train_test_split(dros_strict_real, real_tr_te_name, random_state=seed ,test_set_prop=0.1)
    tr_val_name = 'Drosophila/train_val/dros_tr_val_split_' + str(seed)
    real_train_data, b, c, d = pred.manual_train_test_split(train_data, tr_val_name, random_state=seed, test_set_prop=0.2)

In [28]:
dimensions = [100, 200, 300, 400, 500, 600, 700, 800, 900]
min_word_count = [1, 2, 3, 4, 5, 6, 7, 8, 9]
context_window = [1, 2, 3, 4, 5, 6, 7, 8, 9]
# downsample = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]

In [29]:
# dimensions = [100, 200, 300, 400, 500, 600]
for i in dimensions:
    w2v_parameters = [i, 5, 4, 6, 0.001]
    strict = pred.make_w2v_model(dros_strict_real, 'Drosophila/models/dims/dros_strict_dim_parameter_' + str(i), w2v_parameters)
    gen = pred.make_w2v_model(dros_gen_real, 'Drosophila/models/dims/dros_gen_dim_parameter_' + str(i), w2v_parameters)
    be = pred.make_w2v_model(dros_be_real, 'Drosophila/models/dims/dros_be_dim_parameter_' + str(i), w2v_parameters)

2017-05-28 06:11:20,693 : INFO : collecting all words and their counts
2017-05-28 06:11:20,694 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 06:11:20,729 : INFO : collected 12999 word types from a corpus of 188627 raw words and 7408 sentences
2017-05-28 06:11:20,729 : INFO : Loading a fresh vocabulary
2017-05-28 06:11:20,747 : INFO : min_count=5 retains 3553 unique words (27% of original 12999, drops 9446)
2017-05-28 06:11:20,747 : INFO : min_count=5 leaves 173308 word corpus (91% of original 188627, drops 15319)
2017-05-28 06:11:20,757 : INFO : deleting the raw counts dictionary of 12999 items
2017-05-28 06:11:20,758 : INFO : sample=0.001 downsamples 35 most-common words
2017-05-28 06:11:20,758 : INFO : downsampling leaves estimated 124688 word corpus (71.9% of prior 173308)
2017-05-28 06:11:20,759 : INFO : estimated required memory for 3553 words and 100 dimensions: 6750700 bytes
2017-05-28 06:11:20,764 : INFO : constructing a huffman tree from

Parsing datasets sentences
Training Word2Vec Model


2017-05-28 06:11:20,864 : INFO : built huffman tree with maximum node depth 15
2017-05-28 06:11:20,874 : INFO : resetting layer weights
2017-05-28 06:11:20,914 : INFO : training model with 4 workers on 3553 vocabulary and 100 features, using sg=1 hs=1 sample=0.001 negative=5 window=6
2017-05-28 06:11:20,915 : INFO : expecting 7408 sentences, matching count from corpus used for vocabulary survey
2017-05-28 06:11:21,927 : INFO : PROGRESS: at 43.45% examples, 268481 words/s, in_qsize 8, out_qsize 0
2017-05-28 06:11:22,946 : INFO : PROGRESS: at 85.77% examples, 263822 words/s, in_qsize 7, out_qsize 0
2017-05-28 06:11:23,229 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-28 06:11:23,235 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-28 06:11:23,235 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-28 06:11:23,254 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-28 06:11:23,255 : INFO :

Parsing datasets sentences


2017-05-28 06:11:23,830 : INFO : collecting all words and their counts
2017-05-28 06:11:23,831 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 06:11:23,879 : INFO : PROGRESS: at sentence #10000, processed 238231 words, keeping 12912 word types
2017-05-28 06:11:23,925 : INFO : PROGRESS: at sentence #20000, processed 479313 words, keeping 17503 word types
2017-05-28 06:11:23,971 : INFO : PROGRESS: at sentence #30000, processed 719915 words, keeping 20738 word types
2017-05-28 06:11:24,008 : INFO : collected 22824 word types from a corpus of 904687 raw words and 37697 sentences
2017-05-28 06:11:24,008 : INFO : Loading a fresh vocabulary


...

2017-05-28 06:43:58,810 : INFO : deleting the raw counts dictionary of 22824 items
2017-05-28 06:43:58,812 : INFO : sample=0.001 downsamples 40 most-common words
2017-05-28 06:43:58,812 : INFO : downsampling leaves estimated 644974 word corpus (73.3% of prior 879988)
2017-05-28 06:43:58,813 : INFO : estimated required memory for 8096 words and 900 dimensions: 93104000 bytes
2017-05-28 06:43:58,822 : INFO : constructing a huffman tree from 8096 words
2017-05-28 06:43:58,998 : INFO : built huffman tree with maximum node depth 18
2017-05-28 06:43:59,014 : INFO : resetting layer weights
2017-05-28 06:43:59,182 : INFO : training model with 4 workers on 8096 vocabulary and 900 features, using sg=1 hs=1 sample=0.001 negative=5 window=6
2017-05-28 06:43:59,183 : INFO : expecting 37697 sentences, matching count from corpus used for vocabulary survey
2017-05-28 06:44:00,228 : INFO : PROGRESS: at 1.10% examples, 34180 words/s, in_qsize 8, out_qsize 0
2017-05-28 06:44:01,263 : INFO : PROGRESS: at 

Parsing datasets sentences


2017-05-28 06:45:10,579 : INFO : collecting all words and their counts
2017-05-28 06:45:10,580 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 06:45:10,630 : INFO : PROGRESS: at sentence #10000, processed 222144 words, keeping 13751 word types
2017-05-28 06:45:10,678 : INFO : PROGRESS: at sentence #20000, processed 444687 words, keeping 18858 word types
2017-05-28 06:45:10,727 : INFO : PROGRESS: at sentence #30000, processed 668230 words, keeping 22504 word types
2017-05-28 06:45:10,776 : INFO : PROGRESS: at sentence #40000, processed 890943 words, keeping 25686 word types


Training Word2Vec Model


2017-05-28 06:45:10,825 : INFO : PROGRESS: at sentence #50000, processed 1114050 words, keeping 28248 word types
2017-05-28 06:45:10,874 : INFO : PROGRESS: at sentence #60000, processed 1337267 words, keeping 30428 word types
2017-05-28 06:45:10,923 : INFO : PROGRESS: at sentence #70000, processed 1560562 words, keeping 32404 word types
2017-05-28 06:45:10,972 : INFO : PROGRESS: at sentence #80000, processed 1784789 words, keeping 34244 word types
2017-05-28 06:45:11,020 : INFO : PROGRESS: at sentence #90000, processed 2007797 words, keeping 35969 word types
2017-05-28 06:45:11,070 : INFO : PROGRESS: at sentence #100000, processed 2230870 words, keeping 37530 word types
2017-05-28 06:45:11,119 : INFO : PROGRESS: at sentence #110000, processed 2457061 words, keeping 39108 word types
2017-05-28 06:45:11,168 : INFO : PROGRESS: at sentence #120000, processed 2680877 words, keeping 40469 word types
2017-05-28 06:45:11,217 : INFO : PROGRESS: at sentence #130000, processed 2904497 words, keep

In [30]:
for i in dimensions:
    w2w_model_strict = word2vec.Word2Vec.load('../../Results/Drosophila/models/dims/dros_strict_dim_parameter_'+str(i)+'_model')
    w2w_model_gen = word2vec.Word2Vec.load('../../Results/Drosophila/models/dims/dros_gen_dim_parameter_'+str(i)+'_model')
    w2w_model_be = word2vec.Word2Vec.load('../../Results/Drosophila/models/dims/dros_be_dim_parameter_'+str(i)+'_model')
    
    for seed in random_seeds:
        data_name = '../../Results/Drosophila/train_val/dros_tr_val_split_' + str(seed)
        train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
        train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
        validation_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
        validation_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_strict, feature_count=i)
        
        strict_list_SR_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_gen, feature_count=i)
        
        strict_list_GEN_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                      train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_be, feature_count=i)
        
        strict_list_BE_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        pickle.dump(strict_list_SR_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/dims/dros_strict_list_SR_dims_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_GEN_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/dims/dros_strict_list_GEN_dims_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_BE_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/dims/dros_strict_list_BE_dims_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        
        strict_final_list = [strict_list_SR_dims_param, 
                             strict_list_GEN_dims_param, 
                             strict_list_BE_dims_param]
        print ('\nPredicting\n')
        errors = []
        fpr = []
        tpr = []

        for entry in strict_final_list:
            error_w2v_norm, probs_w2v_norm = pred.XGB_classifier(entry[0], entry[1],
                                                                 entry[2], entry[3])
            fpr_w2v_norm, tpr_w2v_norm, _ = roc_curve(entry[3], probs_w2v_norm)
            error_w2v_fs, probs_w2v_fs = pred.XGB_classifier(entry[0], entry[1],
                                                             entry[2], entry[3],
                                                             feature_selection=True)
            fpr_w2v_fs, tpr_w2v_fs, _ = roc_curve(entry[3], probs_w2v_fs)

            errors.append([error_w2v_norm, error_w2v_fs])
            fpr.append([fpr_w2v_norm, fpr_w2v_fs])
            tpr.append([tpr_w2v_norm, tpr_w2v_fs])
            
        pickle.dump(errors, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/dims/dros_dim_param'+str(i)+'_errors_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(fpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/dims/dros_dim_param'+str(i)+'_fpr_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(tpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/dims/dros_dim_param'+str(i)+'_tpr_pickle_'+str(seed)+'.pkl', 'wb'))

2017-05-28 06:50:58,491 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/dims/dros_strict_dim_parameter_100_model
2017-05-28 06:50:58,536 : INFO : loading wv recursively from ../../Results/Drosophila/models/dims/dros_strict_dim_parameter_100_model.wv.* with mmap=None
2017-05-28 06:50:58,537 : INFO : setting ignored attribute syn0norm to None
2017-05-28 06:50:58,537 : INFO : setting ignored attribute cum_table to None
2017-05-28 06:50:58,538 : INFO : loaded ../../Results/Drosophila/models/dims/dros_strict_dim_parameter_100_model
2017-05-28 06:50:58,548 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/dims/dros_gen_dim_parameter_100_model
2017-05-28 06:50:58,655 : INFO : loading wv recursively from ../../Results/Drosophila/models/dims/dros_gen_dim_parameter_100_model.wv.* with mmap=None
2017-05-28 06:50:58,656 : INFO : setting ignored attribute syn0norm to None
2017-05-28 06:50:58,657 : INFO : setting ignored attribute cum_table to None
2017-05-2


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


In [31]:
def mult_open(direct, pattern):
    pickle_list = []
    file_list = []
    for f in os.listdir(direct):
        if re.search(pattern, f):
            f = f.split('_')
            f[-1] = f[-1][:-4]
            file_list.append(f)
    file_list.sort(key = lambda x: int(x[-1]))
    for file in file_list:
        file = '_'.join(file)
        file = file + '.pkl'
        pkl = pickle.load(open(os.path.join(direct, file), 'rb'))
        pickle_list.append(pkl)
    return pickle_list

In [32]:
best_parameter = []
for seed in random_seeds:
    mean_err_dros_strict = []
    mean_auc_dros_strict = []
    mean_err_dros_gen = []
    mean_auc_dros_gen = []
    mean_err_dros_be = []
    mean_auc_dros_be = []
    for i in dimensions:
        drct = '../../Results/Drosophila/error_fpr_tpr/w2v_param_search/dims/'
        errors_dros = mult_open(drct, str(i)+'_errors_pickle_'+str(seed))
        fpr_dros = mult_open(drct, str(i)+'_fpr_pickle_'+str(seed))
        tpr_dros = mult_open(drct, str(i)+'_tpr_pickle_'+str(seed))
        for e, f, t in zip(errors_dros, fpr_dros, tpr_dros):
            input_list = [[e, f, t]]
            name_list = ['drosophila']
            for idx in range(3):
                for item, name in zip(input_list, name_list):
                    for fpr_item, tpr_item, error_item in zip(item[1][idx], item[2][idx], item[0][idx]):
                        roc_auc = auc(fpr_item, tpr_item)
                        auc_val = '%.3f' % roc_auc
                        error = '%.3f' % error_item
                        if idx == 0:
                            mean_err_dros_strict.append(error_item)
                            mean_auc_dros_strict.append(roc_auc)
                        elif idx == 1:
                            mean_err_dros_gen.append(error_item)
                            mean_auc_dros_gen.append(roc_auc)
                        elif idx == 2:
                            mean_err_dros_be.append(error_item)
                            mean_auc_dros_be.append(roc_auc)

    mean_err_dros_org_strict = mean_err_dros_strict[0::2]
    mean_err_dros_fs_strict = mean_err_dros_strict[1::2]
    mean_auc_dros_org_strict = mean_auc_dros_strict[0::2]
    mean_auc_dros_fs_strict = mean_auc_dros_strict[1::2]

    mean_err_dros_org_gen = mean_err_dros_gen[0::2]
    mean_err_dros_fs_gen = mean_err_dros_gen[1::2]
    mean_auc_dros_org_gen = mean_auc_dros_gen[0::2]
    mean_auc_dros_fs_gen = mean_auc_dros_gen[1::2]

    mean_err_dros_org_be = mean_err_dros_be[0::2]
    mean_err_dros_fs_be = mean_err_dros_be[1::2]
    mean_auc_dros_org_be = mean_auc_dros_be[0::2]
    mean_auc_dros_fs_be = mean_auc_dros_be[1::2]
    
    org_error = max([(mean_err_dros_org_strict.index(min(mean_err_dros_org_strict))+1)*100, 
                     (mean_err_dros_org_gen.index(min(mean_err_dros_org_gen))+1)*100, 
                     (mean_err_dros_org_be.index(min(mean_err_dros_org_be))+1)*100])
    
#     fs_error = max([(mean_err_dros_fs_strict.index(min(mean_err_dros_fs_strict))+1)*100, 
#                      (mean_err_dros_fs_gen.index(min(mean_err_dros_fs_gen))+1)*100, 
#                      (mean_err_dros_fs_be.index(min(mean_err_dros_fs_be))+1)*100])
    
    org_auc = max([(mean_auc_dros_org_strict.index(max(mean_auc_dros_org_strict))+1)*100, 
                     (mean_auc_dros_org_gen.index(max(mean_auc_dros_org_gen))+1)*100, 
                     (mean_auc_dros_org_be.index(max(mean_auc_dros_org_be))+1)*100])
    
#     fs_auc = max([(mean_auc_dros_fs_strict.index(max(mean_auc_dros_fs_strict))+1)*100, 
#                      (mean_auc_dros_fs_gen.index(max(mean_auc_dros_fs_gen))+1)*100, 
#                      (mean_auc_dros_fs_be.index(max(mean_auc_dros_fs_be))+1)*100])
    
    best_parameter.append((seed, [org_error, org_auc]))
    

In [33]:
def most_common(lst):
    return max(set(lst), key=lst.count)

In [34]:
def most_common(lst):
    return max(set(lst), key=lst.count)

best_overall = []
for i in best_parameter:
    best_overall.append(i[1][0])
    best_overall.append(i[1][1])    
    print(i)
best_dims = most_common(best_overall)
print(best_dims)

(144, [900, 900])
(235, [900, 600])
(905, [300, 900])
(2895, [600, 900])
(3462, [700, 800])
(4225, [300, 900])
(5056, [700, 500])
(5192, [300, 700])
(7751, [900, 600])
(7813, [900, 900])
900


In [35]:
# min_word_count = [1, 2, 3, 4, 5, 6]
for i in min_word_count:
    w2v_parameters = [best_dims, i, 4, 6, 0.001]
    strict = pred.make_w2v_model(dros_strict_real, 'Drosophila/models/word_count/dros_strict_word_count_parameter_' + str(i), w2v_parameters)
    gen = pred.make_w2v_model(dros_gen_real, 'Drosophila/models/word_count/dros_gen_word_count_parameter_' + str(i), w2v_parameters)
    be = pred.make_w2v_model(dros_be_real, 'Drosophila/models/word_count/dros_be_word_count_parameter_' + str(i), w2v_parameters)

Parsing datasets sentences


2017-05-28 07:26:22,504 : INFO : collecting all words and their counts
2017-05-28 07:26:22,505 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 07:26:22,538 : INFO : collected 12999 word types from a corpus of 188627 raw words and 7408 sentences
2017-05-28 07:26:22,539 : INFO : Loading a fresh vocabulary
2017-05-28 07:26:22,571 : INFO : min_count=1 retains 12999 unique words (100% of original 12999, drops 0)
2017-05-28 07:26:22,572 : INFO : min_count=1 leaves 188627 word corpus (100% of original 188627, drops 0)
2017-05-28 07:26:22,601 : INFO : deleting the raw counts dictionary of 12999 items
2017-05-28 07:26:22,602 : INFO : sample=0.001 downsamples 32 most-common words
2017-05-28 07:26:22,603 : INFO : downsampling leaves estimated 141308 word corpus (74.9% of prior 188627)
2017-05-28 07:26:22,603 : INFO : estimated required memory for 12999 words and 900 dimensions: 149488500 bytes
2017-05-28 07:26:22,620 : INFO : constructing a huffman tree from 

Training Word2Vec Model


2017-05-28 07:26:22,887 : INFO : built huffman tree with maximum node depth 18
2017-05-28 07:26:22,914 : INFO : resetting layer weights
2017-05-28 07:26:23,161 : INFO : training model with 4 workers on 12999 vocabulary and 900 features, using sg=1 hs=1 sample=0.001 negative=5 window=6
2017-05-28 07:26:23,162 : INFO : expecting 7408 sentences, matching count from corpus used for vocabulary survey
2017-05-28 07:26:24,366 : INFO : PROGRESS: at 5.32% examples, 31251 words/s, in_qsize 8, out_qsize 0
2017-05-28 07:26:25,518 : INFO : PROGRESS: at 13.81% examples, 41396 words/s, in_qsize 7, out_qsize 0
2017-05-28 07:26:26,724 : INFO : PROGRESS: at 22.25% examples, 44183 words/s, in_qsize 7, out_qsize 0
2017-05-28 07:26:28,046 : INFO : PROGRESS: at 30.81% examples, 44462 words/s, in_qsize 7, out_qsize 0
2017-05-28 07:26:29,191 : INFO : PROGRESS: at 38.14% examples, 44707 words/s, in_qsize 7, out_qsize 0
2017-05-28 07:26:30,272 : INFO : PROGRESS: at 43.45% examples, 43173 words/s, in_qsize 7, ou

Parsing datasets sentences


...

2017-05-28 08:25:36,867 : INFO : min_count=9 retains 5720 unique words (25% of original 22824, drops 17104)
2017-05-28 08:25:36,868 : INFO : min_count=9 leaves 865154 word corpus (95% of original 904687, drops 39533)
2017-05-28 08:25:36,884 : INFO : deleting the raw counts dictionary of 22824 items
2017-05-28 08:25:36,885 : INFO : sample=0.001 downsamples 41 most-common words
2017-05-28 08:25:36,886 : INFO : downsampling leaves estimated 628694 word corpus (72.7% of prior 865154)
2017-05-28 08:25:36,887 : INFO : estimated required memory for 5720 words and 900 dimensions: 65780000 bytes
2017-05-28 08:25:36,896 : INFO : constructing a huffman tree from 5720 words
2017-05-28 08:25:37,035 : INFO : built huffman tree with maximum node depth 17
2017-05-28 08:25:37,047 : INFO : resetting layer weights
2017-05-28 08:25:37,192 : INFO : training model with 4 workers on 5720 vocabulary and 900 features, using sg=1 hs=1 sample=0.001 negative=5 window=6
2017-05-28 08:25:37,193 : INFO : expecting 3

Parsing datasets sentences


2017-05-28 08:26:41,315 : INFO : collecting all words and their counts
2017-05-28 08:26:41,316 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 08:26:41,362 : INFO : PROGRESS: at sentence #10000, processed 222144 words, keeping 13751 word types
2017-05-28 08:26:41,403 : INFO : PROGRESS: at sentence #20000, processed 444687 words, keeping 18858 word types
2017-05-28 08:26:41,444 : INFO : PROGRESS: at sentence #30000, processed 668230 words, keeping 22504 word types
2017-05-28 08:26:41,485 : INFO : PROGRESS: at sentence #40000, processed 890943 words, keeping 25686 word types


Training Word2Vec Model


2017-05-28 08:26:41,526 : INFO : PROGRESS: at sentence #50000, processed 1114050 words, keeping 28248 word types
2017-05-28 08:26:41,568 : INFO : PROGRESS: at sentence #60000, processed 1337267 words, keeping 30428 word types
2017-05-28 08:26:41,609 : INFO : PROGRESS: at sentence #70000, processed 1560562 words, keeping 32404 word types
2017-05-28 08:26:41,650 : INFO : PROGRESS: at sentence #80000, processed 1784789 words, keeping 34244 word types
2017-05-28 08:26:41,690 : INFO : PROGRESS: at sentence #90000, processed 2007797 words, keeping 35969 word types
2017-05-28 08:26:41,731 : INFO : PROGRESS: at sentence #100000, processed 2230870 words, keeping 37530 word types
2017-05-28 08:26:41,772 : INFO : PROGRESS: at sentence #110000, processed 2457061 words, keeping 39108 word types
2017-05-28 08:26:41,813 : INFO : PROGRESS: at sentence #120000, processed 2680877 words, keeping 40469 word types
2017-05-28 08:26:41,854 : INFO : PROGRESS: at sentence #130000, processed 2904497 words, keep

In [36]:
for i in min_word_count:
    w2w_model_strict = word2vec.Word2Vec.load('../../Results/Drosophila/models/word_count/dros_strict_word_count_parameter_'+str(i)+'_model')
    w2w_model_gen = word2vec.Word2Vec.load('../../Results/Drosophila/models/word_count/dros_gen_word_count_parameter_'+str(i)+'_model')
    w2w_model_be = word2vec.Word2Vec.load('../../Results/Drosophila/models/word_count/dros_be_word_count_parameter_'+str(i)+'_model')
    
    for seed in random_seeds:
        data_name = '../../Results/Drosophila/train_val/dros_tr_val_split_' + str(seed)
        train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
        train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
        validation_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
        validation_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_strict, feature_count=best_dims)
        
        strict_list_SR_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_gen, feature_count=best_dims)
        
        strict_list_GEN_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                      train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_be, feature_count=best_dims)
        
        strict_list_BE_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        pickle.dump(strict_list_SR_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/word_count/dros_strict_list_SR_word_count_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_GEN_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/word_count/dros_strict_list_GEN_word_count_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_BE_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/word_count/dros_strict_list_BE_word_count_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        
        strict_final_list = [strict_list_SR_dims_param, 
                             strict_list_GEN_dims_param, 
                             strict_list_BE_dims_param]
        print ('\nPredicting\n')
        errors = []
        fpr = []
        tpr = []

        for entry in strict_final_list:
            error_w2v_norm, probs_w2v_norm = pred.XGB_classifier(entry[0], entry[1],
                                                                 entry[2], entry[3])
            fpr_w2v_norm, tpr_w2v_norm, _ = roc_curve(entry[3], probs_w2v_norm)
            error_w2v_fs, probs_w2v_fs = pred.XGB_classifier(entry[0], entry[1],
                                                             entry[2], entry[3],
                                                             feature_selection=True)
            fpr_w2v_fs, tpr_w2v_fs, _ = roc_curve(entry[3], probs_w2v_fs)

            errors.append([error_w2v_norm, error_w2v_fs])
            fpr.append([fpr_w2v_norm, fpr_w2v_fs])
            tpr.append([tpr_w2v_norm, tpr_w2v_fs])
            
        pickle.dump(errors, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/word_count/dros_word_count_param'+str(i)+'_errors_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(fpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/word_count/dros_word_count_param'+str(i)+'_fpr_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(tpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/word_count/dros_word_count_param'+str(i)+'_tpr_pickle_'+str(seed)+'.pkl', 'wb'))

2017-05-28 08:32:19,927 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/word_count/dros_strict_word_count_parameter_1_model
2017-05-28 08:32:20,397 : INFO : loading wv recursively from ../../Results/Drosophila/models/word_count/dros_strict_word_count_parameter_1_model.wv.* with mmap=None
2017-05-28 08:32:20,400 : INFO : loading syn0 from ../../Results/Drosophila/models/word_count/dros_strict_word_count_parameter_1_model.wv.syn0.npy with mmap=None
2017-05-28 08:32:21,025 : INFO : setting ignored attribute syn0norm to None
2017-05-28 08:32:21,026 : INFO : loading syn1neg from ../../Results/Drosophila/models/word_count/dros_strict_word_count_parameter_1_model.syn1neg.npy with mmap=None
2017-05-28 08:32:21,734 : INFO : setting ignored attribute cum_table to None
2017-05-28 08:32:21,735 : INFO : loaded ../../Results/Drosophila/models/word_count/dros_strict_word_count_parameter_1_model
2017-05-28 08:32:21,792 : INFO : loading Word2Vec object from ../../Results/Drosophil


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 M

../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


In [37]:
best_parameter = []
for seed in random_seeds:
    mean_err_dros_strict = []
    mean_auc_dros_strict = []
    mean_err_dros_gen = []
    mean_auc_dros_gen = []
    mean_err_dros_be = []
    mean_auc_dros_be = []
    for i in min_word_count:
        drct = '../../Results/Drosophila/error_fpr_tpr/w2v_param_search/word_count/'
        errors_dros = mult_open(drct, str(i)+'_errors_pickle_'+str(seed))
        fpr_dros = mult_open(drct, str(i)+'_fpr_pickle_'+str(seed))
        tpr_dros = mult_open(drct, str(i)+'_tpr_pickle_'+str(seed))
        for e, f, t in zip(errors_dros, fpr_dros, tpr_dros):
            input_list = [[e, f, t]]
            name_list = ['drosophila']
            for idx in range(3):
                for item, name in zip(input_list, name_list):
                    for fpr_item, tpr_item, error_item in zip(item[1][idx], item[2][idx], item[0][idx]):
                        roc_auc = auc(fpr_item, tpr_item)
                        auc_val = '%.3f' % roc_auc
                        error = '%.3f' % error_item
                        if idx == 0:
                            mean_err_dros_strict.append(error_item)
                            mean_auc_dros_strict.append(roc_auc)
                        elif idx == 1:
                            mean_err_dros_gen.append(error_item)
                            mean_auc_dros_gen.append(roc_auc)
                        elif idx == 2:
                            mean_err_dros_be.append(error_item)
                            mean_auc_dros_be.append(roc_auc)

    mean_err_dros_org_strict = mean_err_dros_strict[0::2]
    mean_err_dros_fs_strict = mean_err_dros_strict[1::2]
    mean_auc_dros_org_strict = mean_auc_dros_strict[0::2]
    mean_auc_dros_fs_strict = mean_auc_dros_strict[1::2]

    mean_err_dros_org_gen = mean_err_dros_gen[0::2]
    mean_err_dros_fs_gen = mean_err_dros_gen[1::2]
    mean_auc_dros_org_gen = mean_auc_dros_gen[0::2]
    mean_auc_dros_fs_gen = mean_auc_dros_gen[1::2]

    mean_err_dros_org_be = mean_err_dros_be[0::2]
    mean_err_dros_fs_be = mean_err_dros_be[1::2]
    mean_auc_dros_org_be = mean_auc_dros_be[0::2]
    mean_auc_dros_fs_be = mean_auc_dros_be[1::2]
    
    org_error = max([(mean_err_dros_org_strict.index(min(mean_err_dros_org_strict))+1), 
                     (mean_err_dros_org_gen.index(min(mean_err_dros_org_gen))+1), 
                     (mean_err_dros_org_be.index(min(mean_err_dros_org_be))+1)])
    
#     fs_error = max([(mean_err_dros_fs_strict.index(min(mean_err_dros_fs_strict))+1), 
#                      (mean_err_dros_fs_gen.index(min(mean_err_dros_fs_gen))+1), 
#                      (mean_err_dros_fs_be.index(min(mean_err_dros_fs_be))+1)])
    
    org_auc = max([(mean_auc_dros_org_strict.index(max(mean_auc_dros_org_strict))+1), 
                     (mean_auc_dros_org_gen.index(max(mean_auc_dros_org_gen))+1), 
                     (mean_auc_dros_org_be.index(max(mean_auc_dros_org_be))+1)])
    
#     fs_auc = max([(mean_auc_dros_fs_strict.index(max(mean_auc_dros_fs_strict))+1), 
#                      (mean_auc_dros_fs_gen.index(max(mean_auc_dros_fs_gen))+1), 
#                      (mean_auc_dros_fs_be.index(max(mean_auc_dros_fs_be))+1)])
    
    best_parameter.append((seed, [org_error, org_auc]))

In [38]:
best_overall = []
for i in best_parameter:
    best_overall.append(i[1][0])
    best_overall.append(i[1][1])    
    print(i)
best_word_count = most_common(best_overall)
print(best_word_count)

(144, [6, 4])
(235, [6, 8])
(905, [7, 8])
(2895, [4, 9])
(3462, [9, 4])
(4225, [5, 7])
(5056, [6, 9])
(5192, [4, 7])
(7751, [3, 9])
(7813, [7, 3])
4


In [39]:
# context_window = [2, 3, 4, 5, 6, 7]
for i in context_window:
    w2v_parameters = [best_dims, best_word_count, 4, i, 0.001]
    strict = pred.make_w2v_model(dros_strict_real, 'Drosophila/models/context_window/dros_strict_context_window_parameter_' + str(i), w2v_parameters)
    gen = pred.make_w2v_model(dros_gen_real, 'Drosophila/models/context_window/dros_gen_context_window_parameter_' + str(i), w2v_parameters)
    be = pred.make_w2v_model(dros_be_real, 'Drosophila/models/context_window/dros_be_context_window_parameter_' + str(i), w2v_parameters)

Parsing datasets sentences


2017-05-28 09:30:15,903 : INFO : collecting all words and their counts
2017-05-28 09:30:15,904 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 09:30:15,963 : INFO : collected 12999 word types from a corpus of 188627 raw words and 7408 sentences
2017-05-28 09:30:15,964 : INFO : Loading a fresh vocabulary


Training Word2Vec Model


2017-05-28 09:30:15,987 : INFO : min_count=4 retains 4197 unique words (32% of original 12999, drops 8802)
2017-05-28 09:30:15,988 : INFO : min_count=4 leaves 175884 word corpus (93% of original 188627, drops 12743)
2017-05-28 09:30:16,007 : INFO : deleting the raw counts dictionary of 12999 items
2017-05-28 09:30:16,009 : INFO : sample=0.001 downsamples 35 most-common words
2017-05-28 09:30:16,010 : INFO : downsampling leaves estimated 127490 word corpus (72.5% of prior 175884)
2017-05-28 09:30:16,011 : INFO : estimated required memory for 4197 words and 900 dimensions: 48265500 bytes
2017-05-28 09:30:16,019 : INFO : constructing a huffman tree from 4197 words
2017-05-28 09:30:16,184 : INFO : built huffman tree with maximum node depth 15
2017-05-28 09:30:16,198 : INFO : resetting layer weights
2017-05-28 09:30:16,355 : INFO : training model with 4 workers on 4197 vocabulary and 900 features, using sg=1 hs=1 sample=0.001 negative=5 window=1
2017-05-28 09:30:16,356 : INFO : expecting 74

Parsing datasets sentences


...

2017-05-28 10:19:51,237 : INFO : min_count=4 retains 9205 unique words (40% of original 22824, drops 13619)
2017-05-28 10:19:51,238 : INFO : min_count=4 leaves 884424 word corpus (97% of original 904687, drops 20263)
2017-05-28 10:19:51,260 : INFO : deleting the raw counts dictionary of 22824 items
2017-05-28 10:19:51,262 : INFO : sample=0.001 downsamples 39 most-common words
2017-05-28 10:19:51,262 : INFO : downsampling leaves estimated 649835 word corpus (73.5% of prior 884424)
2017-05-28 10:19:51,263 : INFO : estimated required memory for 9205 words and 900 dimensions: 105857500 bytes
2017-05-28 10:19:51,275 : INFO : constructing a huffman tree from 9205 words
2017-05-28 10:19:51,478 : INFO : built huffman tree with maximum node depth 18
2017-05-28 10:19:51,496 : INFO : resetting layer weights
2017-05-28 10:19:51,693 : INFO : training model with 4 workers on 9205 vocabulary and 900 features, using sg=1 hs=1 sample=0.001 negative=5 window=9
2017-05-28 10:19:51,694 : INFO : expecting 

Parsing datasets sentences


2017-05-28 10:21:21,638 : INFO : collecting all words and their counts
2017-05-28 10:21:21,639 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 10:21:21,690 : INFO : PROGRESS: at sentence #10000, processed 222144 words, keeping 13751 word types
2017-05-28 10:21:21,739 : INFO : PROGRESS: at sentence #20000, processed 444687 words, keeping 18858 word types
2017-05-28 10:21:21,789 : INFO : PROGRESS: at sentence #30000, processed 668230 words, keeping 22504 word types


Training Word2Vec Model


2017-05-28 10:21:21,839 : INFO : PROGRESS: at sentence #40000, processed 890943 words, keeping 25686 word types
2017-05-28 10:21:21,892 : INFO : PROGRESS: at sentence #50000, processed 1114050 words, keeping 28248 word types
2017-05-28 10:21:21,942 : INFO : PROGRESS: at sentence #60000, processed 1337267 words, keeping 30428 word types
2017-05-28 10:21:21,992 : INFO : PROGRESS: at sentence #70000, processed 1560562 words, keeping 32404 word types
2017-05-28 10:21:22,042 : INFO : PROGRESS: at sentence #80000, processed 1784789 words, keeping 34244 word types
2017-05-28 10:21:22,096 : INFO : PROGRESS: at sentence #90000, processed 2007797 words, keeping 35969 word types
2017-05-28 10:21:22,151 : INFO : PROGRESS: at sentence #100000, processed 2230870 words, keeping 37530 word types
2017-05-28 10:21:22,210 : INFO : PROGRESS: at sentence #110000, processed 2457061 words, keeping 39108 word types
2017-05-28 10:21:22,265 : INFO : PROGRESS: at sentence #120000, processed 2680877 words, keepin

In [40]:
for i in context_window:
    w2w_model_strict = word2vec.Word2Vec.load('../../Results/Drosophila/models/context_window/dros_strict_context_window_parameter_'+str(i)+'_model')
    w2w_model_gen = word2vec.Word2Vec.load('../../Results/Drosophila/models/context_window/dros_gen_context_window_parameter_'+str(i)+'_model')
    w2w_model_be = word2vec.Word2Vec.load('../../Results/Drosophila/models/context_window/dros_be_context_window_parameter_'+str(i)+'_model')
    
    for seed in random_seeds:
        data_name = '../../Results/Drosophila/train_val/dros_tr_val_split_' + str(seed)
        train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
        train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
        validation_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
        validation_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_strict, feature_count=best_dims)
        
        strict_list_SR_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_gen, feature_count=best_dims)
        
        strict_list_GEN_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                      train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_be, feature_count=best_dims)
        
        strict_list_BE_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        pickle.dump(strict_list_SR_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/context_window/dros_strict_list_SR_context_window_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_GEN_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/context_window/dros_strict_list_GEN_context_window_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_BE_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/context_window/dros_strict_list_BE_context_window_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        
        strict_final_list = [strict_list_SR_dims_param, 
                             strict_list_GEN_dims_param, 
                             strict_list_BE_dims_param]
        print ('\nPredicting\n')
        errors = []
        fpr = []
        tpr = []

        for entry in strict_final_list:
            error_w2v_norm, probs_w2v_norm = pred.XGB_classifier(entry[0], entry[1],
                                                                 entry[2], entry[3])
            fpr_w2v_norm, tpr_w2v_norm, _ = roc_curve(entry[3], probs_w2v_norm)
            error_w2v_fs, probs_w2v_fs = pred.XGB_classifier(entry[0], entry[1],
                                                             entry[2], entry[3],
                                                             feature_selection=True)
            fpr_w2v_fs, tpr_w2v_fs, _ = roc_curve(entry[3], probs_w2v_fs)

            errors.append([error_w2v_norm, error_w2v_fs])
            fpr.append([fpr_w2v_norm, fpr_w2v_fs])
            tpr.append([tpr_w2v_norm, tpr_w2v_fs])
            
        pickle.dump(errors, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/context_window/dros_context_window_param'+str(i)+'_errors_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(fpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/context_window/dros_context_window_param'+str(i)+'_fpr_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(tpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/context_window/dros_context_window_param'+str(i)+'_tpr_pickle_'+str(seed)+'.pkl', 'wb'))

2017-05-28 10:28:59,647 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/context_window/dros_strict_context_window_parameter_1_model
2017-05-28 10:29:00,779 : INFO : loading wv recursively from ../../Results/Drosophila/models/context_window/dros_strict_context_window_parameter_1_model.wv.* with mmap=None
2017-05-28 10:29:00,780 : INFO : setting ignored attribute syn0norm to None
2017-05-28 10:29:00,780 : INFO : setting ignored attribute cum_table to None
2017-05-28 10:29:00,781 : INFO : loaded ../../Results/Drosophila/models/context_window/dros_strict_context_window_parameter_1_model
2017-05-28 10:29:00,794 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/context_window/dros_gen_context_window_parameter_1_model
2017-05-28 10:29:02,773 : INFO : loading wv recursively from ../../Results/Drosophila/models/context_window/dros_gen_context_window_parameter_1_model.wv.* with mmap=None
2017-05-28 10:29:02,775 : INFO : setting ignored attribute syn0norm


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


In [41]:
best_parameter = []
for seed in random_seeds:
    mean_err_dros_strict = []
    mean_auc_dros_strict = []
    mean_err_dros_gen = []
    mean_auc_dros_gen = []
    mean_err_dros_be = []
    mean_auc_dros_be = []
    for i in context_window:
        drct = '../../Results/Drosophila/error_fpr_tpr/w2v_param_search/context_window/'
        errors_dros = mult_open(drct, str(i)+'_errors_pickle_'+str(seed))
        fpr_dros = mult_open(drct, str(i)+'_fpr_pickle_'+str(seed))
        tpr_dros = mult_open(drct, str(i)+'_tpr_pickle_'+str(seed))
        for e, f, t in zip(errors_dros, fpr_dros, tpr_dros):
            input_list = [[e, f, t]]
            name_list = ['drosophila']
            for idx in range(3):
                for item, name in zip(input_list, name_list):
                    for fpr_item, tpr_item, error_item in zip(item[1][idx], item[2][idx], item[0][idx]):
                        roc_auc = auc(fpr_item, tpr_item)
                        auc_val = '%.3f' % roc_auc
                        error = '%.3f' % error_item
                        if idx == 0:
                            mean_err_dros_strict.append(error_item)
                            mean_auc_dros_strict.append(roc_auc)
                        elif idx == 1:
                            mean_err_dros_gen.append(error_item)
                            mean_auc_dros_gen.append(roc_auc)
                        elif idx == 2:
                            mean_err_dros_be.append(error_item)
                            mean_auc_dros_be.append(roc_auc)

    mean_err_dros_org_strict = mean_err_dros_strict[0::2]
    mean_err_dros_fs_strict = mean_err_dros_strict[1::2]
    mean_auc_dros_org_strict = mean_auc_dros_strict[0::2]
    mean_auc_dros_fs_strict = mean_auc_dros_strict[1::2]

    mean_err_dros_org_gen = mean_err_dros_gen[0::2]
    mean_err_dros_fs_gen = mean_err_dros_gen[1::2]
    mean_auc_dros_org_gen = mean_auc_dros_gen[0::2]
    mean_auc_dros_fs_gen = mean_auc_dros_gen[1::2]

    mean_err_dros_org_be = mean_err_dros_be[0::2]
    mean_err_dros_fs_be = mean_err_dros_be[1::2]
    mean_auc_dros_org_be = mean_auc_dros_be[0::2]
    mean_auc_dros_fs_be = mean_auc_dros_be[1::2]
    
    org_error = max([(mean_err_dros_org_strict.index(min(mean_err_dros_org_strict))+1), 
                     (mean_err_dros_org_gen.index(min(mean_err_dros_org_gen))+1), 
                     (mean_err_dros_org_be.index(min(mean_err_dros_org_be))+1)])
    
#     fs_error = max([(mean_err_dros_fs_strict.index(min(mean_err_dros_fs_strict))+2), 
#                      (mean_err_dros_fs_gen.index(min(mean_err_dros_fs_gen))+2), 
#                      (mean_err_dros_fs_be.index(min(mean_err_dros_fs_be))+2)])
    
    org_auc = max([(mean_auc_dros_org_strict.index(max(mean_auc_dros_org_strict))+1), 
                     (mean_auc_dros_org_gen.index(max(mean_auc_dros_org_gen))+1), 
                     (mean_auc_dros_org_be.index(max(mean_auc_dros_org_be))+1)])
    
#     fs_auc = max([(mean_auc_dros_fs_strict.index(max(mean_auc_dros_fs_strict))+2), 
#                      (mean_auc_dros_fs_gen.index(max(mean_auc_dros_fs_gen))+2), 
#                      (mean_auc_dros_fs_be.index(max(mean_auc_dros_fs_be))+2)])
    
    best_parameter.append((seed, [org_error, org_auc]))

In [42]:
best_overall = []
for i in best_parameter:
    best_overall.append(i[1][0])
    best_overall.append(i[1][1])    
    print(i)
best_context_window = most_common(best_overall)
print(best_context_window)

(144, [9, 9])
(235, [4, 9])
(905, [7, 9])
(2895, [9, 9])
(3462, [4, 4])
(4225, [8, 7])
(5056, [9, 6])
(5192, [8, 3])
(7751, [9, 9])
(7813, [9, 8])
9


In [43]:
downsample = [(1, 0.1), (2, 0.01), (3, 0.001), (4, 0.0001), (5, 0.00001), (6, 0.000001)]
for i in downsample:
    w2v_parameters = [best_dims, best_word_count, 4, best_context_window, i[1]]
    strict = pred.make_w2v_model(dros_strict_real, 'Drosophila/models/downsample/dros_strict_downsample_parameter_' + str(i[0]), w2v_parameters)
    gen = pred.make_w2v_model(dros_gen_real, 'Drosophila/models/downsample/dros_gen_downsample_parameter_' + str(i[0]), w2v_parameters)
    be = pred.make_w2v_model(dros_be_real, 'Drosophila/models/downsample/dros_be_downsample_parameter_' + str(i[0]), w2v_parameters)

Parsing datasets sentences


2017-05-28 11:35:56,392 : INFO : collecting all words and their counts
2017-05-28 11:35:56,393 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 11:35:56,429 : INFO : collected 12999 word types from a corpus of 188627 raw words and 7408 sentences
2017-05-28 11:35:56,430 : INFO : Loading a fresh vocabulary


Training Word2Vec Model


2017-05-28 11:35:56,605 : INFO : min_count=4 retains 4197 unique words (32% of original 12999, drops 8802)
2017-05-28 11:35:56,606 : INFO : min_count=4 leaves 175884 word corpus (93% of original 188627, drops 12743)
2017-05-28 11:35:56,617 : INFO : deleting the raw counts dictionary of 12999 items
2017-05-28 11:35:56,619 : INFO : sample=0.1 downsamples 0 most-common words
2017-05-28 11:35:56,619 : INFO : downsampling leaves estimated 175884 word corpus (100.0% of prior 175884)
2017-05-28 11:35:56,620 : INFO : estimated required memory for 4197 words and 900 dimensions: 48265500 bytes
2017-05-28 11:35:56,625 : INFO : constructing a huffman tree from 4197 words
2017-05-28 11:35:56,713 : INFO : built huffman tree with maximum node depth 15
2017-05-28 11:35:56,724 : INFO : resetting layer weights
2017-05-28 11:35:56,809 : INFO : training model with 4 workers on 4197 vocabulary and 900 features, using sg=1 hs=1 sample=0.1 negative=5 window=9
2017-05-28 11:35:56,810 : INFO : expecting 7408 s

Parsing datasets sentences


...

2017-05-28 12:20:15,994 : INFO : deleting the raw counts dictionary of 22824 items
2017-05-28 12:20:15,996 : INFO : sample=1e-06 downsamples 9205 most-common words
2017-05-28 12:20:15,997 : INFO : downsampling leaves estimated 57883 word corpus (6.5% of prior 884424)
2017-05-28 12:20:15,997 : INFO : estimated required memory for 9205 words and 900 dimensions: 105857500 bytes
2017-05-28 12:20:16,008 : INFO : constructing a huffman tree from 9205 words
2017-05-28 12:20:16,191 : INFO : built huffman tree with maximum node depth 18
2017-05-28 12:20:16,210 : INFO : resetting layer weights
2017-05-28 12:20:16,382 : INFO : training model with 4 workers on 9205 vocabulary and 900 features, using sg=1 hs=1 sample=1e-06 negative=5 window=9
2017-05-28 12:20:16,383 : INFO : expecting 37697 sentences, matching count from corpus used for vocabulary survey
2017-05-28 12:20:17,391 : INFO : PROGRESS: at 28.48% examples, 81920 words/s, in_qsize 6, out_qsize 1
2017-05-28 12:20:18,399 : INFO : PROGRESS: a

Parsing datasets sentences


2017-05-28 12:20:22,919 : INFO : collecting all words and their counts
2017-05-28 12:20:22,920 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 12:20:22,966 : INFO : PROGRESS: at sentence #10000, processed 222144 words, keeping 13751 word types
2017-05-28 12:20:23,003 : INFO : PROGRESS: at sentence #20000, processed 444687 words, keeping 18858 word types
2017-05-28 12:20:23,041 : INFO : PROGRESS: at sentence #30000, processed 668230 words, keeping 22504 word types
2017-05-28 12:20:23,079 : INFO : PROGRESS: at sentence #40000, processed 890943 words, keeping 25686 word types
2017-05-28 12:20:23,117 : INFO : PROGRESS: at sentence #50000, processed 1114050 words, keeping 28248 word types


Training Word2Vec Model


2017-05-28 12:20:23,155 : INFO : PROGRESS: at sentence #60000, processed 1337267 words, keeping 30428 word types
2017-05-28 12:20:23,193 : INFO : PROGRESS: at sentence #70000, processed 1560562 words, keeping 32404 word types
2017-05-28 12:20:23,232 : INFO : PROGRESS: at sentence #80000, processed 1784789 words, keeping 34244 word types
2017-05-28 12:20:23,271 : INFO : PROGRESS: at sentence #90000, processed 2007797 words, keeping 35969 word types
2017-05-28 12:20:23,308 : INFO : PROGRESS: at sentence #100000, processed 2230870 words, keeping 37530 word types
2017-05-28 12:20:23,347 : INFO : PROGRESS: at sentence #110000, processed 2457061 words, keeping 39108 word types
2017-05-28 12:20:23,385 : INFO : PROGRESS: at sentence #120000, processed 2680877 words, keeping 40469 word types
2017-05-28 12:20:23,428 : INFO : PROGRESS: at sentence #130000, processed 2904497 words, keeping 41802 word types
2017-05-28 12:20:23,468 : INFO : PROGRESS: at sentence #140000, processed 3129297 words, kee

In [44]:
downsample = [(1, 0.1), (2, 0.01), (3, 0.001), (4, 0.0001), (5, 0.00001), (6, 0.000001)]

In [45]:
for i in downsample:
    w2w_model_strict = word2vec.Word2Vec.load('../../Results/Drosophila/models/downsample/dros_strict_downsample_parameter_'+str(i[0])+'_model')
    w2w_model_gen = word2vec.Word2Vec.load('../../Results/Drosophila/models/downsample/dros_gen_downsample_parameter_'+str(i[0])+'_model')
    w2w_model_be = word2vec.Word2Vec.load('../../Results/Drosophila/models/downsample/dros_be_downsample_parameter_'+str(i[0])+'_model')
    
    for seed in random_seeds:
        data_name = '../../Results/Drosophila/train_val/dros_tr_val_split_' + str(seed)
        train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
        train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
        validation_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
        validation_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_strict, feature_count=best_dims)
        
        strict_list_SR_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_gen, feature_count=best_dims)
        
        strict_list_GEN_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                      train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_be, feature_count=best_dims)
        
        strict_list_BE_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        pickle.dump(strict_list_SR_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/downsample/dros_strict_list_SR_downsample_param_'+str(i[0])+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_GEN_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/downsample/dros_strict_list_GEN_downsample_param_'+str(i[0])+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_BE_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/downsample/dros_strict_list_BE_downsample_param_'+str(i[0])+'_'+str(seed)+'_results_list.pkl', 'wb'))
        
        strict_final_list = [strict_list_SR_dims_param, 
                             strict_list_GEN_dims_param, 
                             strict_list_BE_dims_param]
        print ('\nPredicting\n')
        errors = []
        fpr = []
        tpr = []

        for entry in strict_final_list:
            error_w2v_norm, probs_w2v_norm = pred.XGB_classifier(entry[0], entry[1],
                                                                 entry[2], entry[3])
            fpr_w2v_norm, tpr_w2v_norm, _ = roc_curve(entry[3], probs_w2v_norm)
            error_w2v_fs, probs_w2v_fs = pred.XGB_classifier(entry[0], entry[1],
                                                             entry[2], entry[3],
                                                             feature_selection=True)
            fpr_w2v_fs, tpr_w2v_fs, _ = roc_curve(entry[3], probs_w2v_fs)

            errors.append([error_w2v_norm, error_w2v_fs])
            fpr.append([fpr_w2v_norm, fpr_w2v_fs])
            tpr.append([tpr_w2v_norm, tpr_w2v_fs])
            
        pickle.dump(errors, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/dros_downsample_param'+str(i[0])+'_errors_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(fpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/dros_downsample_param'+str(i[0])+'_fpr_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(tpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/dros_downsample_param'+str(i[0])+'_tpr_pickle_'+str(seed)+'.pkl', 'wb'))

2017-05-28 12:20:54,762 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/downsample/dros_strict_downsample_parameter_1_model
2017-05-28 12:20:54,956 : INFO : loading wv recursively from ../../Results/Drosophila/models/downsample/dros_strict_downsample_parameter_1_model.wv.* with mmap=None
2017-05-28 12:20:54,957 : INFO : setting ignored attribute syn0norm to None
2017-05-28 12:20:54,958 : INFO : setting ignored attribute cum_table to None
2017-05-28 12:20:54,958 : INFO : loaded ../../Results/Drosophila/models/downsample/dros_strict_downsample_parameter_1_model
2017-05-28 12:20:54,972 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/downsample/dros_gen_downsample_parameter_1_model
2017-05-28 12:20:55,370 : INFO : loading wv recursively from ../../Results/Drosophila/models/downsample/dros_gen_downsample_parameter_1_model.wv.* with mmap=None
2017-05-28 12:20:55,371 : INFO : setting ignored attribute syn0norm to None
2017-05-28 12:20:55,372 : INFO 


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


In [46]:
best_parameter = []
for seed in random_seeds:
    mean_err_dros_strict = []
    mean_auc_dros_strict = []
    mean_err_dros_gen = []
    mean_auc_dros_gen = []
    mean_err_dros_be = []
    mean_auc_dros_be = []
    for i in downsample:
        drct = '../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/'
        errors_dros = mult_open(drct, str(i[0])+'_errors_pickle_'+str(seed))
        fpr_dros = mult_open(drct, str(i[0])+'_fpr_pickle_'+str(seed))
        tpr_dros = mult_open(drct, str(i[0])+'_tpr_pickle_'+str(seed))
        for e, f, t in zip(errors_dros, fpr_dros, tpr_dros):
            input_list = [[e, f, t]]
            name_list = ['drosophila']
            for idx in range(3):
                for item, name in zip(input_list, name_list):
                    for fpr_item, tpr_item, error_item in zip(item[1][idx], item[2][idx], item[0][idx]):
                        roc_auc = auc(fpr_item, tpr_item)
                        auc_val = '%.3f' % roc_auc
                        error = '%.3f' % error_item
                        if idx == 0:
                            mean_err_dros_strict.append(error_item)
                            mean_auc_dros_strict.append(roc_auc)
                        elif idx == 1:
                            mean_err_dros_gen.append(error_item)
                            mean_auc_dros_gen.append(roc_auc)
                        elif idx == 2:
                            mean_err_dros_be.append(error_item)
                            mean_auc_dros_be.append(roc_auc)

    mean_err_dros_org_strict = mean_err_dros_strict[0::2]
    mean_err_dros_fs_strict = mean_err_dros_strict[1::2]
    mean_auc_dros_org_strict = mean_auc_dros_strict[0::2]
    mean_auc_dros_fs_strict = mean_auc_dros_strict[1::2]

    mean_err_dros_org_gen = mean_err_dros_gen[0::2]
    mean_err_dros_fs_gen = mean_err_dros_gen[1::2]
    mean_auc_dros_org_gen = mean_auc_dros_gen[0::2]
    mean_auc_dros_fs_gen = mean_auc_dros_gen[1::2]

    mean_err_dros_org_be = mean_err_dros_be[0::2]
    mean_err_dros_fs_be = mean_err_dros_be[1::2]
    mean_auc_dros_org_be = mean_auc_dros_be[0::2]
    mean_auc_dros_fs_be = mean_auc_dros_be[1::2]
    
    org_error = max([(mean_err_dros_org_strict.index(min(mean_err_dros_org_strict))), 
                     (mean_err_dros_org_gen.index(min(mean_err_dros_org_gen))), 
                     (mean_err_dros_org_be.index(min(mean_err_dros_org_be)))])
    
#     fs_error = max([(mean_err_dros_fs_strict.index(min(mean_err_dros_fs_strict))), 
#                      (mean_err_dros_fs_gen.index(min(mean_err_dros_fs_gen))), 
#                      (mean_err_dros_fs_be.index(min(mean_err_dros_fs_be)))])
    
    org_auc = max([(mean_auc_dros_org_strict.index(max(mean_auc_dros_org_strict))), 
                     (mean_auc_dros_org_gen.index(max(mean_auc_dros_org_gen))), 
                     (mean_auc_dros_org_be.index(max(mean_auc_dros_org_be)))])
    
#     fs_auc = max([(mean_auc_dros_fs_strict.index(max(mean_auc_dros_fs_strict))), 
#                      (mean_auc_dros_fs_gen.index(max(mean_auc_dros_fs_gen))), 
#                      (mean_auc_dros_fs_be.index(max(mean_auc_dros_fs_be)))])
    
    best_parameter.append((seed, [org_error, org_auc]))

In [47]:
best_overall = []
for i in best_parameter:
    best_overall.append(i[1][0])
    best_overall.append(i[1][1])    
    print(i)
best_downsample = most_common(best_overall)
print(best_downsample)

(144, [1, 5])
(235, [5, 2])
(905, [5, 5])
(2895, [4, 3])
(3462, [4, 3])
(4225, [3, 5])
(5056, [5, 5])
(5192, [2, 5])
(7751, [1, 1])
(7813, [4, 5])
5


In [27]:
best_w2v_parameters = [best_dims, best_word_count, 4, best_context_window, downsample[best_downsample-1][1]]
print(best_w2v_parameters)

[800, 8, 4, 8, 0.001]


In [48]:
best_w2v_parameters = [best_dims, best_word_count, 4, best_context_window, downsample[best_downsample-1][1]]
print(best_w2v_parameters)

[900, 4, 4, 9, 1e-05]


0.1
0.01
0.001
0.0001
1e-05
1e-06


In [56]:
print(mean_auc_dros_org_be)

[0.49761245674740484, 0.47173010380622837, 0.5404844290657439, 0.48937716262975778, 0.54927335640138408, 0.50200692041522488]


In [69]:
w2v_parameters = [900, 4, 4, 9, 1e-05]
strict = pred.make_w2v_model(dros_strict_real, 'Drosophila/models/downsample/dros_strict_model', w2v_parameters)
gen = pred.make_w2v_model(dros_gen_real, 'Drosophila/models/downsample/dros_gen_model', w2v_parameters)
be = pred.make_w2v_model(dros_be_real, 'Drosophila/models/downsample/dros_be_model', w2v_parameters)

2017-05-28 15:58:28,162 : INFO : collecting all words and their counts
2017-05-28 15:58:28,163 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 15:58:28,197 : INFO : collected 12999 word types from a corpus of 188627 raw words and 7408 sentences
2017-05-28 15:58:28,197 : INFO : Loading a fresh vocabulary
2017-05-28 15:58:28,216 : INFO : min_count=4 retains 4197 unique words (32% of original 12999, drops 8802)
2017-05-28 15:58:28,217 : INFO : min_count=4 leaves 175884 word corpus (93% of original 188627, drops 12743)
2017-05-28 15:58:28,232 : INFO : deleting the raw counts dictionary of 12999 items
2017-05-28 15:58:28,233 : INFO : sample=1e-05 downsamples 3553 most-common words
2017-05-28 15:58:28,233 : INFO : downsampling leaves estimated 31511 word corpus (17.9% of prior 175884)
2017-05-28 15:58:28,234 : INFO : estimated required memory for 4197 words and 900 dimensions: 48265500 bytes
2017-05-28 15:58:28,239 : INFO : constructing a huffman tree fr

Parsing datasets sentences
Training Word2Vec Model


2017-05-28 15:58:28,328 : INFO : built huffman tree with maximum node depth 15
2017-05-28 15:58:28,339 : INFO : resetting layer weights
2017-05-28 15:58:28,434 : INFO : training model with 4 workers on 4197 vocabulary and 900 features, using sg=1 hs=1 sample=1e-05 negative=5 window=9
2017-05-28 15:58:28,435 : INFO : expecting 7408 sentences, matching count from corpus used for vocabulary survey
2017-05-28 15:58:29,473 : INFO : PROGRESS: at 40.22% examples, 61613 words/s, in_qsize 7, out_qsize 0
2017-05-28 15:58:30,474 : INFO : PROGRESS: at 77.32% examples, 59817 words/s, in_qsize 7, out_qsize 0
2017-05-28 15:58:30,975 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-28 15:58:30,991 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-28 15:58:31,013 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-28 15:58:31,040 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-28 15:58:31,041 : INFO : t

Parsing datasets sentences


2017-05-28 15:58:32,242 : INFO : collecting all words and their counts
2017-05-28 15:58:32,243 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 15:58:32,287 : INFO : PROGRESS: at sentence #10000, processed 238231 words, keeping 12912 word types
2017-05-28 15:58:32,330 : INFO : PROGRESS: at sentence #20000, processed 479313 words, keeping 17503 word types
2017-05-28 15:58:32,374 : INFO : PROGRESS: at sentence #30000, processed 719915 words, keeping 20738 word types
2017-05-28 15:58:32,411 : INFO : collected 22824 word types from a corpus of 904687 raw words and 37697 sentences
2017-05-28 15:58:32,412 : INFO : Loading a fresh vocabulary


...

2017-05-28 15:58:32,635 : INFO : min_count=4 retains 9205 unique words (40% of original 22824, drops 13619)
2017-05-28 15:58:32,636 : INFO : min_count=4 leaves 884424 word corpus (97% of original 904687, drops 20263)
2017-05-28 15:58:32,663 : INFO : deleting the raw counts dictionary of 22824 items
2017-05-28 15:58:32,665 : INFO : sample=1e-05 downsamples 3124 most-common words
2017-05-28 15:58:32,666 : INFO : downsampling leaves estimated 188244 word corpus (21.3% of prior 884424)
2017-05-28 15:58:32,666 : INFO : estimated required memory for 9205 words and 900 dimensions: 105857500 bytes
2017-05-28 15:58:32,682 : INFO : constructing a huffman tree from 9205 words
2017-05-28 15:58:32,880 : INFO : built huffman tree with maximum node depth 18
2017-05-28 15:58:32,902 : INFO : resetting layer weights
2017-05-28 15:58:33,092 : INFO : training model with 4 workers on 9205 vocabulary and 900 features, using sg=1 hs=1 sample=1e-05 negative=5 window=9
2017-05-28 15:58:33,093 : INFO : expectin

Parsing datasets sentences


2017-05-28 15:58:57,595 : INFO : collecting all words and their counts
2017-05-28 15:58:57,596 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-28 15:58:57,643 : INFO : PROGRESS: at sentence #10000, processed 222144 words, keeping 13751 word types
2017-05-28 15:58:57,692 : INFO : PROGRESS: at sentence #20000, processed 444687 words, keeping 18858 word types
2017-05-28 15:58:57,740 : INFO : PROGRESS: at sentence #30000, processed 668230 words, keeping 22504 word types
2017-05-28 15:58:57,791 : INFO : PROGRESS: at sentence #40000, processed 890943 words, keeping 25686 word types


Training Word2Vec Model


2017-05-28 15:58:57,841 : INFO : PROGRESS: at sentence #50000, processed 1114050 words, keeping 28248 word types
2017-05-28 15:58:57,892 : INFO : PROGRESS: at sentence #60000, processed 1337267 words, keeping 30428 word types
2017-05-28 15:58:57,939 : INFO : PROGRESS: at sentence #70000, processed 1560562 words, keeping 32404 word types
2017-05-28 15:58:57,990 : INFO : PROGRESS: at sentence #80000, processed 1784789 words, keeping 34244 word types
2017-05-28 15:58:58,038 : INFO : PROGRESS: at sentence #90000, processed 2007797 words, keeping 35969 word types
2017-05-28 15:58:58,087 : INFO : PROGRESS: at sentence #100000, processed 2230870 words, keeping 37530 word types
2017-05-28 15:58:58,138 : INFO : PROGRESS: at sentence #110000, processed 2457061 words, keeping 39108 word types
2017-05-28 15:58:58,185 : INFO : PROGRESS: at sentence #120000, processed 2680877 words, keeping 40469 word types
2017-05-28 15:58:58,233 : INFO : PROGRESS: at sentence #130000, processed 2904497 words, keep

In [70]:
best_dims = w2v_parameters[0]
for seed in random_seeds:
    data_name = '../../Results/Drosophila/train_test/dros_tr_te_split_' + str(seed)
    train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
    train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
    validation_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
    validation_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))

    w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, strict, feature_count=best_dims)

    strict_list_SR_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                 train_labels, validation_labels]

    w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, gen, feature_count=best_dims)

    strict_list_GEN_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                  train_labels, validation_labels]

    w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, be, feature_count=best_dims)

    strict_list_BE_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                 train_labels, validation_labels]

    pickle.dump(strict_list_SR_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/downsample/dros_strict_list_SR_model_'+str(seed)+'_results_list.pkl', 'wb'))
    pickle.dump(strict_list_GEN_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/downsample/dros_strict_list_GEN_model_'+str(seed)+'_results_list.pkl', 'wb'))
    pickle.dump(strict_list_BE_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/downsample/dros_strict_list_BE_model_'+str(seed)+'_results_list.pkl', 'wb'))

    strict_final_list = [strict_list_SR_dims_param, 
                         strict_list_GEN_dims_param, 
                         strict_list_BE_dims_param]
    print ('\nPredicting\n')
    errors = []
    fpr = []
    tpr = []

    for entry in strict_final_list:
        error_w2v_norm, probs_w2v_norm = pred.XGB_classifier(entry[0], entry[1],
                                                             entry[2], entry[3])
        fpr_w2v_norm, tpr_w2v_norm, _ = roc_curve(entry[3], probs_w2v_norm)
        error_w2v_fs, probs_w2v_fs = pred.XGB_classifier(entry[0], entry[1],
                                                         entry[2], entry[3],
                                                         feature_selection=True)
        fpr_w2v_fs, tpr_w2v_fs, _ = roc_curve(entry[3], probs_w2v_fs)

        errors.append([error_w2v_norm, error_w2v_fs])
        fpr.append([fpr_w2v_norm, fpr_w2v_fs])
        tpr.append([tpr_w2v_norm, tpr_w2v_fs])

    pickle.dump(errors, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/dros_model_errors_pickle_'+str(seed)+'.pkl', 'wb'))
    pickle.dump(fpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/dros_model_fpr_pickle_'+str(seed)+'.pkl', 'wb'))
    pickle.dump(tpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/dros_model_tpr_pickle_'+str(seed)+'.pkl', 'wb'))

../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


In [71]:
best_parameter = []
for seed in random_seeds:
    mean_err_dros_strict = []
    mean_auc_dros_strict = []
    mean_err_dros_gen = []
    mean_auc_dros_gen = []
    mean_err_dros_be = []
    mean_auc_dros_be = []
    for i in range(1):
        drct = '../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/'
        errors_dros = mult_open(drct,'model_errors_pickle_'+str(seed))
        fpr_dros = mult_open(drct, 'model_fpr_pickle_'+str(seed))
        tpr_dros = mult_open(drct, 'model_tpr_pickle_'+str(seed))
        for e, f, t in zip(errors_dros, fpr_dros, tpr_dros):
            input_list = [[e, f, t]]
            name_list = ['drosophila']
            for idx in range(3):
                for item, name in zip(input_list, name_list):
                    for fpr_item, tpr_item, error_item in zip(item[1][idx], item[2][idx], item[0][idx]):
                        roc_auc = auc(fpr_item, tpr_item)
                        auc_val = '%.3f' % roc_auc
                        error = '%.3f' % error_item
                        if idx == 0:
                            mean_err_dros_strict.append(error_item)
                            mean_auc_dros_strict.append(roc_auc)
                        elif idx == 1:
                            mean_err_dros_gen.append(error_item)
                            mean_auc_dros_gen.append(roc_auc)
                        elif idx == 2:
                            mean_err_dros_be.append(error_item)
                            mean_auc_dros_be.append(roc_auc)

    mean_err_dros_org_strict = mean_err_dros_strict[0::2]
    mean_err_dros_fs_strict = mean_err_dros_strict[1::2]
    mean_auc_dros_org_strict = mean_auc_dros_strict[0::2]
    mean_auc_dros_fs_strict = mean_auc_dros_strict[1::2]

    mean_err_dros_org_gen = mean_err_dros_gen[0::2]
    mean_err_dros_fs_gen = mean_err_dros_gen[1::2]
    mean_auc_dros_org_gen = mean_auc_dros_gen[0::2]
    mean_auc_dros_fs_gen = mean_auc_dros_gen[1::2]

    mean_err_dros_org_be = mean_err_dros_be[0::2]
    mean_err_dros_fs_be = mean_err_dros_be[1::2]
    mean_auc_dros_org_be = mean_auc_dros_be[0::2]
    mean_auc_dros_fs_be = mean_auc_dros_be[1::2]
    
    org_error = max([(mean_err_dros_org_strict.index(min(mean_err_dros_org_strict))), 
                     (mean_err_dros_org_gen.index(min(mean_err_dros_org_gen))), 
                     (mean_err_dros_org_be.index(min(mean_err_dros_org_be)))])
    
#     fs_error = max([(mean_err_dros_fs_strict.index(min(mean_err_dros_fs_strict))), 
#                      (mean_err_dros_fs_gen.index(min(mean_err_dros_fs_gen))), 
#                      (mean_err_dros_fs_be.index(min(mean_err_dros_fs_be)))])
    
    org_auc = max([(mean_auc_dros_org_strict.index(max(mean_auc_dros_org_strict))), 
                     (mean_auc_dros_org_gen.index(max(mean_auc_dros_org_gen))), 
                     (mean_auc_dros_org_be.index(max(mean_auc_dros_org_be)))])
    
#     fs_auc = max([(mean_auc_dros_fs_strict.index(max(mean_auc_dros_fs_strict))), 
#                      (mean_auc_dros_fs_gen.index(max(mean_auc_dros_fs_gen))), 
#                      (mean_auc_dros_fs_be.index(max(mean_auc_dros_fs_be)))])
    
    best_parameter.append((seed, [org_error, org_auc]))

In [68]:
print(mean_auc_dros_org_strict, mean_auc_dros_org_gen, mean_auc_dros_org_be)
print(mean_err_dros_org_strict, mean_err_dros_org_gen, mean_err_dros_org_be)

[0.6826923076923076] [0.58653846153846145] [0.45192307692307687]
[0.17021276595744683] [0.17021276595744683] [0.17021276595744683]


In [72]:
print(mean_auc_dros_org_strict, mean_auc_dros_org_gen, mean_auc_dros_org_be)
print(mean_err_dros_org_strict, mean_err_dros_org_gen, mean_err_dros_org_be)

[0.58974358974358976] [0.49358974358974361] [0.44230769230769229]
[0.17021276595744683] [0.17021276595744683] [0.17021276595744683]
